In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [21]:
# parameters
tensorboard_dir = '../tb/run6';
tensorboardFlag = True;
workers = 10;
histogram_freq = 0;

batchSize = 64;
epochs = 10;
validation_size=0.3;

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
df_train = pd.read_csv('../input/labels.csv')
df_test = pd.read_csv('../input/sample_submission.csv')

print('Training images: ',df_train.shape[0])
print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

Training images:  10222
Test images:  10357


In [4]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
im_size = 90

In [8]:
x_train = []
y_train = []
x_test = []

In [9]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

100%|██████████| 10222/10222 [00:30<00:00, 329.82it/s]


In [10]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread('../input/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 10357/10357 [00:30<00:00, 338.46it/s]


In [11]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(10222, 90, 90, 3)
(10222, 120)
(10357, 90, 90, 3)


We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
num_class = y_train_raw.shape[1]
print('Number of classes: ', num_class)

Number of classes:  120


It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [22]:
# Create the base pre-trained model
# Can't download weights in the kernel
base_model = VGG19(#weights='imagenet',
    weights = 'imagenet', include_top=False, input_shape=(im_size, im_size, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

callbacks_list = [];
callbacks_list.append(keras.callbacks.EarlyStopping(
    monitor='val_acc',
    patience=10,
    verbose=1));
if tensorboardFlag:
    callbacks_list.append(keras.callbacks.TensorBoard(
            log_dir=tensorboard_dir,
            histogram_freq=histogram_freq,
            write_graph=False,
            write_images=False));
    print('Tensorboard activated in directory: ',tensorboard_dir)
else:
    print('Tensorboard NOT activated')


model.summary()

Tensorboard activated in directory:  ../tb/run5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 90, 90, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 90, 90, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 90, 90, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 45, 45, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 45, 45, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 45, 45, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)  

In [16]:
def generator(X,Y,batch_size):
    batch_features = np.ndarray(shape=(batch_size,) + X.shape[1:],
                                dtype=X.dtype);
    batch_labels = np.ndarray(shape=(batch_size,) + Y.shape[1:],
                                dtype=Y.dtype)
    N = X.shape[0];
    
    while True:
        for i in range(batch_size):
            # choose random index in features
            index= np.random.choice(N,1)
            batch_features[i] = X[index]
            batch_labels[i] = Y[index]
        yield batch_features, batch_labels

In [23]:
model.fit(X_train, Y_train,
          epochs=epochs,
          batch_size = batchSize,
          validation_data=(X_valid, Y_valid),
          verbose=1,
          callbacks=callbacks_list)

# steps_per_epoch = round(X_train.shape[0]/batchSize)
# model.fit_generator(generator(X_train,Y_train,batchSize),
#                    steps_per_epoch=steps_per_epoch,
#                    epochs=epochs,
#                    verbose=1,
#                    callbacks=callbacks_list,
#                    validation_data=(X_valid,Y_valid),
#                    workers=workers,
#                    use_multiprocessing=True)

Train on 7155 samples, validate on 3067 samples
Epoch 1/10
7155/7155 [==============================] - 26s 4ms/step - loss: 4.6696 - acc: 0.0335 - val_loss: 4.3520 - val_acc: 0.0796
Epoch 2/10
7155/7155 [==============================] - 26s 4ms/step - loss: 3.8368 - acc: 0.1625 - val_loss: 4.1430 - val_acc: 0.1047
Epoch 3/10
7155/7155 [==============================] - 26s 4ms/step - loss: 3.3782 - acc: 0.2639 - val_loss: 4.0664 - val_acc: 0.1223
Epoch 4/10
7155/7155 [==============================] - 26s 4ms/step - loss: 3.0476 - acc: 0.3318 - val_loss: 4.0290 - val_acc: 0.1219
Epoch 5/10
7155/7155 [==============================] - 26s 4ms/step - loss: 2.7775 - acc: 0.4015 - val_loss: 3.9747 - val_acc: 0.1298
Epoch 6/10
7155/7155 [==============================] - 26s 4ms/step - loss: 2.5441 - acc: 0.4461 - val_loss: 3.9599 - val_acc: 0.1301
Epoch 7/10
7155/7155 [==============================] - 26s 4ms/step - loss: 2.3439 - acc: 0.5086 - val_loss: 3.9737 - val_acc: 0.1376
Epoch 8

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [24]:
preds = model.predict(x_test, verbose=1)

10357/10357 [==============================] - 28s 3ms/step


In [25]:
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(10)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,8.977614e-06,2.058467e-03,2.953424e-06,2.834425e-06,0.004311,0.000052,0.000126,0.000711,9.409663e-03,...,1.317757e-03,1.539885e-04,5.363414e-05,9.527878e-04,1.954524e-02,1.299639e-01,9.260310e-07,4.768775e-04,0.000126,0.000015
1,00102ee9d8eb90812350685311fe5890,2.763812e-04,1.731115e-03,8.001035e-05,3.014813e-05,0.000612,0.000114,0.000769,0.000116,2.744072e-04,...,6.950337e-03,1.230091e-02,4.714187e-05,6.241740e-05,7.707609e-04,3.571663e-04,3.518358e-03,5.854042e-04,0.011657,0.000119
2,0012a730dfa437f5f3613fb75efcd4ce,9.695714e-04,1.602765e-03,1.461310e-03,1.056984e-02,0.007570,0.028564,0.001623,0.043498,8.097608e-03,...,4.663540e-03,2.657343e-03,7.329178e-04,5.075794e-03,1.405053e-02,3.178776e-03,3.529280e-03,3.620130e-02,0.017505,0.000583
3,001510bc8570bbeee98c8d80c8a95ec1,8.924165e-03,6.765326e-03,1.105086e-03,1.577807e-02,0.004862,0.019543,0.007722,0.155049,2.511635e-04,...,8.854185e-04,1.092843e-03,3.146500e-04,8.243831e-05,3.443761e-04,5.538823e-04,8.408279e-05,2.469887e-02,0.000872,0.001031
4,001a5f3114548acdefa3d4da05474c2e,1.389753e-02,4.035940e-03,1.732581e-02,2.388310e-02,0.008297,0.001924,0.014864,0.000883,1.027024e-03,...,1.505542e-03,3.238849e-04,4.034016e-04,1.780710e-04,8.360330e-03,7.936951e-05,6.398355e-04,4.345995e-03,0.009893,0.001115
5,00225dcd3e4d2410dd53239f95c0352f,3.819395e-05,2.545938e-02,2.052132e-04,5.738481e-04,0.001550,0.000006,0.002255,0.002402,5.480761e-03,...,1.053314e-02,1.214514e-03,7.797933e-04,1.367094e-02,4.235110e-03,5.755645e-03,3.034843e-04,3.940601e-02,0.003088,0.000306
6,002c2a3117c2193b4d26400ce431eebd,4.625614e-03,8.327835e-03,8.268281e-03,9.772742e-03,0.002929,0.007508,0.125620,0.012112,2.738851e-04,...,1.404368e-03,1.609720e-02,2.285474e-03,3.432672e-03,1.077524e-03,1.321773e-02,7.555991e-03,6.096500e-04,0.006207,0.025170
7,002c58d413a521ae8d1a5daeb35fc803,1.927418e-08,1.348139e-07,1.738515e-09,2.050532e-07,0.000163,0.000001,0.000494,0.000092,5.894461e-09,...,4.972010e-07,4.928291e-07,1.229494e-11,3.053390e-09,5.256471e-12,8.646798e-07,3.023320e-01,9.443148e-07,0.060565,0.000003
8,002f80396f1e3db687c5932d7978b196,2.960608e-03,2.341126e-03,6.554633e-02,8.635166e-03,0.008728,0.004145,0.014262,0.008312,9.197900e-04,...,1.253495e-03,2.590940e-04,6.784584e-04,5.207917e-04,2.943838e-02,1.336569e-03,8.311726e-04,1.350470e-02,0.004099,0.000675
9,0036c6bcec6031be9e62a257b1c3c442,1.921794e-03,3.187023e-04,1.248891e-03,2.359620e-04,0.001231,0.003022,0.001636,0.000133,8.008133e-04,...,2.011388e-04,4.754839e-05,3.559669e-04,2.859689e-03,2.753715e-01,5.790440e-04,1.759257e-05,2.306983e-04,0.000296,0.000083
